# Project convex optimization: oligomerization of proteins

*Selected Topics in Mathematical Optimization: 2017-2018*

**Michiel Stock** ([email](michiel.stock@ugent.be))

![](Figures/logo.png)

$X = += t \delta X$

$X = np.max(X, 10^{-10})$

to make sure that concentrations are positive

YOUR NAME(S) HERE

In [1]:
import numpy as np
import matplotlib.pyplot as plt 

%matplotlib inline

In this project we will study the association of protein complexes through the principle of **entropy maximization**. 

Consider the following three proteins: A, B and C:

![Three monomeric proteins.](Figures/monomers.jpg)

Each of these proteins has sites which it can use to reversible bind to other proteins. Two types of interactions can be distinguished:
- **weak type I interactions**: protein A has one type I donor site, protein C has one type I acceptor site and protein B has both an acceptor and a donor site. In the figures, these sites are indicated as a triangle sticking in (acceptor) or out (donor) the protein.
- **strong type II interaction**: protein A has one type II acceptor site and protein C has one type II donor site. These sites are depicted as two bumps sticking out or pressing in the protein.

These sites allow for dimerisation of the proteins. All possible homo- and hetero-dimers are shown below.

![The proteins can form dimers according to their sites.](Figures/dimers.jpg)

But tri-and tetramers can also be formed:

![One trimer and three tetramers.](Figures/oligomers.jpg)

Let us use one (monomer), two (dimer), three (trimer) and four (tetramer) character long strings to denote all possible conformations. Here we use the convention that the string $P_1P_2P_3P_4$ represents the tetramer
$$
\left (\begin{array}{cc}
P_1 & P_2 \\
P_4 & P_3\end{array} 
\right ) \,.
$$

Note that in our notation, we have
$$
P_1P_2P_3P_4 = P_4P_1P_2P_3 = P_3P_4P_1P_2 = P_2P_3P_4P_1\,,
$$
but
$$
P_1P_2P_3 \neq P_3P_1P_2 \neq P_2P_3P_1\,.
$$

The following code identifies all unique mono-and oligomers.

In [2]:
monomers = set(['A', 'B', 'C'])

dimers = set(['AC', 'CB', 'CA', 'BA', 'BB'])

trimers = set([dm1 + dm2[1] for dm1 in dimers
               for dm2 in dimers if dm1[1:] == dm2[:-1]])

tetramers = set([dm1 + dm2[2] for dm1 in trimers
                 for dm2 in trimers if dm1[1:] == dm2[:-1]
                 if dm2[-1] + dm1[0] in dimers])

# some tetramers are counted multiple times, because our encoding is not unique
# let us remove duplicates

lowest_lexo = lambda polymer : sorted([polymer[i:] + polymer[:i] for i in range(len(polymer))])[0]
tetramers = set(map(lowest_lexo, tetramers))

compounds = monomers | dimers | trimers | tetramers
compounds = list(compounds)
compounds.sort()
monomers =list(monomers)
monomers.sort()

print('There are {} unique compounds'.format(len(compounds)))

There are 19 unique compounds


In [3]:
monomers

['A', 'B', 'C']

In [4]:
compounds

['A',
 'AC',
 'ACA',
 'ACAC',
 'ACB',
 'ACBB',
 'B',
 'BA',
 'BAC',
 'BB',
 'BBA',
 'BBB',
 'BBBB',
 'C',
 'CA',
 'CAC',
 'CB',
 'CBA',
 'CBB']

Oligiomerzation is a process that releases energy. The change of heat in a system is quantified by the enthalpy. Formation of a type I bonds results in a change of enthalphy of $\Delta H$ of -1 Joule / mol (heat is released when two monomers bind), while type II bonds have a change of enthalpy of $\Delta H$ of -2 Joule / mol.

The following piece of code returns the energy the formation enthalpy of a compound.

In [5]:
binding_enthalpy = {'AC' : -2, 'BA' : -1, 'BB' : -1, 'CA' : -1, 'CB' : -1}

def get_enthalpy(compound):
    if len(compound) == 1:
        return 0  # no bonds
    else:
        enthalpy = 0
        for i in range(len(compound) - 1):
            dimer = compound[i:i+2]
            enthalpy += binding_enthalpy[dimer]
        if len(compound)==4:
            enthalpy += binding_enthalpy[compound[-1] + compound[0]]
        return enthalpy

In [6]:
enthalpies_dict = {compound : get_enthalpy(compound) for compound in compounds}

print('Formation enthalpies:')
for compound, enthalpy in enthalpies_dict.items():
    print('{} => {}'.format(compound, enthalpy))
    
enthalpies = [enthalpies_dict[comp] for comp in compounds]  # as list, same order as compounds
enthalpies = np.array(enthalpies).reshape((-1, 1))  # as Numpy array

Formation enthalpies:
ACB => -3
ACAC => -6
AC => -2
BBB => -2
BB => -1
B => 0
BAC => -3
ACBB => -5
CBA => -2
CAC => -3
CA => -1
BA => -1
CB => -1
A => 0
BBA => -2
ACA => -3
CBB => -2
C => 0
BBBB => -4


Since all the association and dissociation reactions are assumed to be reversible, a mixture of the three monomers should give rise to all possible mono- and oligomers (though not necessarily in equal quantities). We will assume that the system will go to an equilibrium where the Gibbs free engery is highest. The Gibbs free energy depends both on the entropy as well as the enthalpy of the system.

Let us denote $x$ as the vector containing the concentrations of the 19 possible species. The entropy of the system is given by

$$
S(\mathbf{x}) = - \sum_{i=1}^{19}x_i \log x_i\,.
$$

The enthalpy of the system is given by

$$
H(\mathbf{x}) = \sum_{i=1}^{19}x_i h_i\,,
$$

with $h_i$ the formation enthalpy of compound $i$.

The Gibbs free energy is then given by

$$
G(\mathbf{x}) = H(\mathbf{x}) - T S(\mathbf{x})
$$

with $T\geq0$ the temperature of the system (in Kelvin). Systems with a constant temperture go to a state with a minimal Gibbs free energy.

Note that:
- By low temperatures, enthalpy dominates. The enthalpy can be raised by the formation of oligomers with multiple low-energy bonds.
- By high temperature, entropy dominates. High entropy can be obtained by having many different species at a low concentration.

The concentration of each species can not be freely chosen, vector $x$ has two types of constraints:
- **equality constraints**: there is a conservation of mass: the total quantity of A, B, C in all the species should remain constant. These form four linear equality contraints.
- **inequality constraints**: since $x$ is a vector with concentrations all elements should be larger than zero: i.e. $x_i \geq 0$. 

The equality constraints are given by the stoichiometric matrix $S$, a $3\times 19$ matrix which quantifies how many of individual molecules $A$, $B$ and $C$ are in a complex and $c$ is the vector of length 3 containing the total of $A$, $B$ and $C$ in the system.

The optimization problem can thus be formulated as follows:

$$
\min_\mathbf{x} G(\mathbf{x}) = H(\mathbf{x}) - T S(\mathbf{x})
$$
$$
\text{subject to }  \mathbf{x} \succeq 0
$$
$$
S\mathbf{x}=\mathbf{c}\,.
$$

In this project you will have to find the equilibrium concentrations by minimizing the Gibbs free energy by a given temperature.

> NOTE: We have chosen **not** to explicitly add the inequality constraints in the optimization problem (i.e. using the logaritmic barrier function). The entropy term will ensure that a minimizer of the Gibbs free energy will always have strictly positive values of $\mathbf{x}$, i.e. concentrations greater than 0. This can easily be seen by noting that $\partial S(\mathbf{x})/\partial x_i|_{0}=-\infty$. To circumvent numerical problems, at the end of every Newton step, we will enforce feasibility by making sure that every component of $\mathbf{x}$ is greater than some small positive tolerance ($10^{-10}$).

** Project assignments**

1. Suppose that an initial mixture contains only the monomers A, B and C with relative fractions of 0.3, 0.4 and 0.3. Give the matrix $S$ and vector $\mathbf{c}$ of the linear constraints of the system.
2. First assume $T=0$ (no entropy term). In this case the problem is a linear programming problem. Find the equilibrium mixture with the lowest energy. You can use the linear programming solver in `scipy`: `from scipy.optimize import linprog`. What compounds are formed?
3. Compute the first and second-order partial derivatives of the Gibbs free energy. Complete the functions `gibbs_fun`, `gibbs_grad` and `gibbs_hessian` which give the function value, gradient and hessian of the Gibbs free energy. These functions take as inputs the vector of concentrations $\mathbf{x}$ and the temperature $T$. The enthalpy per mol is available in the vector `enthalpies`.
4. Complete the implementation for performing linearly constrained Newton step. This function should do a step which satifies the mass balance equations.
5. Complete the implementation of `newton_oligiomerization`. Make sure that after every step the concentratons are strictly positive. Use this to calculate the equilibrium concentration of the different species at a temperature of 100 Kelvin. **Check if it satifies the mass balance!**.
6. Make a plot of the concentrations, enthalpy and entropy of the equilibrium system at different temperatures. Vary $T$ from $ 10^{-1},\ldots, 10^3$. Describe what you see.
7. Use grid search (step size of 0.05) to find the initial quantities of A, B and C (total quantity is equal to 1, at least 0.05 mol/L of each momomer) that have to be mixed at a temperature of 100K to obain:
    - the equilibrium mixture with the highest entropy
    - the equilibrium mixture with the lowest enthalphy
    - the equilibrium mixture with the highest concentration of CBA

**ASSIGNMENT 1**

Complete the matrix $S$ and vector $\mathbf{c}$ to describe the mass balance of the system.

In [7]:
# your code here!
S = np.array([list(map(lambda j: j.count(thechar), compounds)) for thechar in monomers])
c = np.array([[.3, .4, .3]]).T

In [8]:
print(S)

[[1 1 2 2 1 1 0 1 1 0 1 0 0 0 1 1 0 1 0]
 [0 0 0 0 1 2 1 1 1 2 2 3 4 0 0 0 1 1 2]
 [0 1 1 2 1 1 0 0 1 0 0 0 0 1 1 2 1 1 1]]


In [9]:
print(c)

[[ 0.3]
 [ 0.4]
 [ 0.3]]


**ASSIGNMENT 2**

Use linear programming to find the equilibrium solution at $T=0K$.

In [10]:
from scipy.optimize import linprog

In [11]:
linprog?

In [12]:
xstar_0K = linprog(list(map(lambda y:get_enthalpy(y), compounds)), A_eq=S, b_eq=c)['x']

In [13]:
xstar_0K

array([ 0.  ,  0.  ,  0.  ,  0.15,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.1 ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ])

In [14]:
np.array(compounds)[xstar_0K>0]

array(['ACAC', 'BBBB'], 
      dtype='<U4')

DESCRIBE THE RESULT

**ASSIGNMENT 3**

Complete the function and partial derivatives and implement these functions.

$$
\min_{\mathbf{x}} \sum_{i=1}^{19}[h_ix_i + Tx_i \log x_i]
$$
$$
\text{subject to } S\mathbf{x}=\mathbf{c}\,.
$$

**Function**

$$
f(\mathbf{x}, T)=\ldots
$$

**Gradient**

$$
\frac{\partial f(\mathbf{x}, T)}{\partial x_i}=\ldots
$$

$$
\nabla f(\mathbf{x}, T)=\ldots
$$

**Hessian**

$$
\frac{\partial^2 f(\mathbf{x}, T)}{\partial x_i^2}=\ldots
$$

Note that

$$
\frac{\partial^2 f(\mathbf{x}, T)}{\partial x_i\partial x_j}=\ldots \qquad \text{if $i\neq j$}
$$

In [15]:
def gibbs_fun(x, T, h):
    """
    Negative entropy
    Inputs:
        - x: vector of the concentrations (19 x 1)
        - T: temperature

    Output: the function value of the negative entropy
    """
    return np.sum(h@x + T*x@np.log(x))

def gibbs_gradient(x, T):
    """
    Gradient of the negative entropy
    Inputs:
        - x: vector of the concentrations (19 x 1)
        - T: temperature

    Output: the gradient of the negative entropy
        (19 x 1 vector)
    """
    return (np.atleast_2d(list(map(lambda y:get_enthalpy(y), compounds))).T + T*(np.log(x)+1))

def gibbs_hessian(x, T):
    """
    Hessian of negative entropy
    Inputs:
        - x: vector of the concentrations (19 x 1)
        - T: temperature

    Output: the gradient of the negative entropy
        (19 x 19 matrix)
    """
    return np.eye(x.shape[0])*np.divide(T,x)

**ASSIGNMENT 4**

Complete the code for a single constrained Newton step for this problem.

In [21]:
def solve_constrained_quadratic_problem(P, q, A, b):
    """
    Solve a linear constrained quadratic convex problem.

    Inputs:
        - P, q: quadratic and linear parameters of
                the linear function to be minimized
        - A, b: system of the linear constraints

    Outputs:
        - xstar: the exact minimizer
        - vstar: the optimal Lagrange multipliers
    """
    p, n = A.shape  # size of the problem
    # complete this code
    # HINT: use np.linalg.solve and np.bmat
    
    
    pdb.set_trace()
    solution = np.linalg.solve(np.bmat([[P, A.T], [A, np.zeros((p,p))]]),
                               np.bmat([[-q], [b]]))
    xstar = solution[:n]
    vstar = solution[n:]
    return np.array(xstar), np.array(vstar)

In [ ]:
def constained_newton_step(x, S, c, T):
    """
    Computes a constrained Newton step for the
    Gibbs free energy minimization problem. 
    
    Note: 
    
    Inputs:
        - x: point in which to compute the Newton step, does not have to be feasible
        - S, c: matrix and vector of the system describing the mass balance
        - T: temperature of the system
        
    Output:
        - Dx: the Newton step
    """
    # complete
    ddfx = gibbs_hessian(x, T)
    dfx = gibbs_gradient(x, T)
    Dx, _ = solve_constrained_quadratic_problem(ddfx, dfx, S, c - S @ x) # complete!

    return Dx

In [18]:
import pdb

In [24]:
# test step (does it make any sense?)
X = np.ones((19,1))/10
T = 100
solve_constrained_quadratic_problem(X, S, c, T)

> <ipython-input-21-d60051a02f7e>(20)solve_constrained_quadratic_problem()
-> solution = np.linalg.solve(np.bmat([[P, A.T], [A, np.zeros((p,p))]]),
(Pdb) P
array([[ 0.1],
       [ 0.1],
       [ 0.1],
       [ 0.1],
       [ 0.1],
       [ 0.1],
       [ 0.1],
       [ 0.1],
       [ 0.1],
       [ 0.1],
       [ 0.1],
       [ 0.1],
       [ 0.1],
       [ 0.1],
       [ 0.1],
       [ 0.1],
       [ 0.1],
       [ 0.1],
       [ 0.1]])
(Pdb) c


ValueError: all the input array dimensions except for the concatenation axis must match exactly

**ASSIGNMENT 5**

Complete the code for Newton's method for finding the optimal concentration vector.

In [ ]:
def newton_oligomerization(x0, S, c, T, stepsize, tolerance=1e-10, epsilon=1e-7):
    """
    Newton's method for finding concentrations that minimize the Gibbs free energy
    at a specific temperature
    
    Inputs:
        - x0: starting point, vector with strictly positive elements,
                    but not necessarily feasible
        - S, c: matrix and vector of the system describing the mass balance
        - T: temperature of the system
        - stepsize: fixed step size
        - tolerance: tolerance parameter to make the vectors strictly feasible
            after every step, minimum value of every element in x
        - epsilon: tolerance for stopping
        
    Output:
        - x: minimizer
    """
    x = x0
    while True:
        Dx = ...  # compute search direction
        if ...:  # determine convergence
            break
        # update x
        # make x feasible
    return x

In [ ]:
# starting points should contain positive concentrations
# but not necessarily satisfy mass balance
x0 = np.ones(...)

In [ ]:
# Compute the equilibrium concentrations

xstar_100K = ...

In [ ]:
# check if solution satisfies mass balance
...

**Assignment 6**

In [ ]:
# functions for calculating entropy and enthalpy
entropy = lambda x : - np.sum(x * np.log(x))
enthalpy = lambda x : np.sum(enthalpies * x.reshape((-1, 1)))

In [ ]:
# compute the obtained concentrations, entropy en enthalpy at different tempartures

In [ ]:
# plot the results

**Describe the obtained plots and give a physicochemical interpretation**

YOUR TEXT HERE

**Assignment 7**

In [ ]:
# Use grid search to find initial mixtures of A, B and C that give rise to mixtures
# with the highest entropy, lowest entropy and highest concentration of CBA at 100K.

**Find inital mixture which will lead to the highest entropy.**

In [ ]:
# your code here

**Find inital mixture which will lead to the lowest enthalpy.**

In [ ]:
# your code here

**Find inital mixture which will lead to the highest concentration CBA.**

In [ ]:
# your code here

## References

Boyd, S. and Vandenberghe, L., '*[Convex Optimization](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf)*'. Cambridge University Press (2004)